In [1]:
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np
from functools import reduce

In [5]:
# Load in all the datasets which we have to combine
df_akamai = pd.read_csv('../raw_data/akamai_server_count.tsv', sep='\t') # ['Service', 'Total', 'City', 'State', 'Country', 'Continent', '1', '0']
df_akamai = df_akamai.groupby(['Country'], as_index=True).agg(Akai_Server_Count=('Total','sum'), Lat= ('1','mean'), Long=('0','mean'))

df_internet_access = pd.read_csv('../raw_data/WORLD_BANK_INTERNET_ACCESS_DATA.csv') # ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', years:'1960' - '2020']
df_population = pd.read_csv('../raw_data/WORLD_BANK_POPULATION_DATA.csv') # ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', years:'1960' - '2020']

df_population = df_population.rename(columns={'Country Name': 'Country'})
df_population = df_population.set_index('Country')
df_population = df_population['2017']

# We have to pivot these tables
df_annual_co2_by_country = pd.read_csv('../raw_data/annual-co2-emissions-per-country.csv') # ['Entity', 'Code', 'Year', 'Annual CO2 emissions'] -> multiple entries for many years for each country
df_annual_co2_emissions_per_capita = pd.read_csv('../raw_data/co-emissions-per-capita.csv') # ['Entity', 'Code', 'Year', 'Per capita CO2 emissions']
df_energy = pd.read_csv('../raw_data/energy.csv') # ['Entity', 'Code', 'Year', 'Primary energy consumption (TWh)']
df_share_renewables = pd.read_csv('../raw_data/share-electricity-renewables.csv') # ['Entity', 'Code', 'Year', 'Renewables (% electricity)']
df_share_using_internet = pd.read_csv('../raw_data/share-of-individuals-using-the-internet.csv') # ['Entity', 'Code', 'Year', 'Individuals using the Internet (% of population)']

def process_world_in_data_dataset(data, colName, year):
  # Rename the entity columns so that they also say country
  data = data.rename(columns={'Entity': 'Country'})
  
  # First we convert the world of data tables to have one row for each country and change the years to columns
  # Set the index as the Entity (Country) -> and do the same for the other tables
  data = data.pivot(index='Country', columns="Year", values=colName)

  # House work / clean up with the indices
  data.rename_axis(None).reset_index(drop=True)

  # Keep only the last 10 years of data for each dataset
  # valid_years = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
  data = data[year]
  return data

df_annual_co2_by_country = process_world_in_data_dataset(df_annual_co2_by_country, 'Annual CO2 emissions', 2017)
df_annual_co2_emissions_per_capita = process_world_in_data_dataset(df_annual_co2_emissions_per_capita, 'Per capita CO2 emissions', 2017)
df_energy = process_world_in_data_dataset(df_energy, 'Primary energy consumption (TWh)', 2017)
df_share_renewables = process_world_in_data_dataset(df_share_renewables, 'Renewables (% electricity)', 2017)
df_share_using_internet = process_world_in_data_dataset(df_share_using_internet, 'Individuals using the Internet (% of population)', 2017)

dfs = [df_akamai, df_population, df_annual_co2_by_country, df_annual_co2_emissions_per_capita, df_energy, df_share_renewables, df_share_using_internet]
df = reduce(lambda left,right: pd.merge(left,right,on='Country', how='inner'), dfs)
df.columns = ['akamai_server_count', 'lat', 'long', 'population', 'annual_co2_by_country', 'annual_co2_emissions_per_capita', 'energy', 'share_renewables', 'share_using_internet']
df.to_csv('../golden_source.csv')

/var/folders/5p/3492pb752gx28hp2p2vhhh9h0000gn/T/ipykernel_2163/730671058.py:42: FutureWarning: Passing 'suffixes' which cause duplicate columns {'2017_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = reduce(lambda left,right: pd.merge(left,right,on='Country', how='inner'), dfs)
